# Expected value and the gradient of parameters
In this example, we show how to calculate the expected value of a parameterized quantum circuit and the way to extract the graident of each parameters via Ted-Q module and pyTorch module

In [ ]:
import tedq as qai
import torch
import numpy as np

In [ ]:
n_qubits=3

### Circuit definition

In [ ]:
def circuitDef(theta):
    # Circuit
    qai.RX(theta[0], qubits=[0])
    qai.RY(theta[1], qubits=[1])
    qai.RZ(theta[2], qubits=[2])
    
    qai.CNOT(qubits=[0, 1])
    qai.CNOT(qubits=[1, 2])
    qai.CNOT(qubits=[2, 0])
    
    # Measurement
    for idx in range(n_qubits):
        qai.measurement.expval(qai.PauliZ(qubits=[idx]))

### Parse the circuit

In [ ]:
circuit = qai.Circuit(circuitDef, n_qubits, torch.rand(n_qubits))

### Draw the circuit

In [ ]:
drawer = qai.matplotlib_drawer(circuit, dpi=100, figsize=(5,6))
drawer.full_draw()

### Compile the circuit

In [ ]:
my_compilecircuit = circuit.compilecircuit(backend="pytorch" )

### init the parameters

In [ ]:
params = torch.rand(n_qubits, requires_grad=True)

### calculate the expected value of the circuit

In [ ]:
exp_val = my_compilecircuit(params)

### calculate the gradient

In [ ]:
optimizer = torch.optim.Adam([params], lr=0.5)
loss = torch.nn.MSELoss(reduction='mean')
l = loss(exp_val, torch.Tensor([0, 1, 1]))
l.backward()

### Print the result

In [ ]:
print("Initial parameters: ", params)
print("Expected value from circuit: ", exp_val)
print("Gradient of parameters", params.grad)